#### Converter o modelo para o formato ONNX e usar uma pipeline otimizada

Essa etapa tem como objetivo converter o modelo para o formato ONNX, que é usado para exportar modelos treinados para aplicações em produção. Esse formato é leve, otimizado e facilita a integração com diferentes frameworks e ambientes. No entanto, devido às limitações de recursos computacionais mencionadas na etapa de treinamento, não consegui realizar a conversão neste caso. Mesmo assim, decidi incluir o conceito no projeto para mostrar sua relevância e destacar como ele pode ser explorado em cenários futuros.


In [2]:
import cv2
import glob
import torch
import numpy as np
from PIL import Image
from super_gradients.training import models

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CLASSES = ['product']
CLASSES += [str(i) for i in range(80 - len(CLASSES))]

model = models.get(
    "yolo_nas_s",
    num_classes=len(CLASSES),
    checkpoint_path=f"./weights/SKU110K/RUN_20241215_173503_278811/average_model.pth"
)

In [3]:
models.convert_to_onnx(model=model, input_shape=(3,640,640), out_path="./weights/SKU110K/RUN_20241215_173503_278811/average_model.onnx")

'./weights/SKU110K/RUN_20241215_173503_278811/average_model.onnx'

In [4]:
from yolo_nas_onnx.models import load_net
from yolo_nas_onnx.processing import Preprocessing, Postprocessing
from yolo_nas_onnx.draw import draw_box
from yolo_nas_onnx.utils import Labels

In [5]:
def detect(net, source, pre_process, post_process, labels):
    net_input = source.copy()  
    input_, prep_meta = pre_process(net_input)  
    outputs = net.forward(input_) 

    boxes, scores, classes = post_process(outputs, prep_meta)  
    selected = cv2.dnn.NMSBoxes(
        boxes, scores, post_process.score_thres, post_process.iou_thres
    ) 

    for i in selected:  
        box = boxes[i, :].astype(np.int32).flatten()  
        score = float(scores[i]) * 100  
        label, color = labels(classes[i], use_bgr=True) 

        draw_box(source, box, label, score, color) 
    return source 

use_gpu = True
use_opencv_dnn_runtime = False
model_path = "./weights/SKU110K/RUN_20241215_173503_278811/average_model.onnx"

net = load_net(model_path, use_gpu, use_opencv_dnn_runtime)
net.assert_input_shape([1,3,640,640])
net.warmup()

RuntimeError: D:\a\_work\1\s\onnxruntime\python\onnxruntime_pybind_state.cc:562 onnxruntime::python::CreateExecutionProviderInstance CUDA_PATH is set but CUDA wasn't able to be loaded. Please install the correct version of CUDA and cuDNN as mentioned in the GPU requirements page (https://onnxruntime.ai/docs/reference/execution-providers/CUDA-ExecutionProvider.html#requirements), make sure they're in the PATH, and that your GPU is supported.


In [ ]:
prep_steps = [
    {"DetLongMaxRescale": None},
    {"BotRightPad": {"pad_value": 114}}
]

iou_thres = 0.65
score_thres = 0.5
labels = ["0"]

_, _, input_height, input_width = net.input_shape  # get input height and width [b, c, h, w]

pre_process = Preprocessing(
    prep_steps, (input_height, input_width)
)

post_process = Postprocessing(
    prep_steps,
    iou_thres,
    score_thres,
)

labels = Labels(labels)

In [ ]:
import cv2
import numpy as np
from PIL import Image

img = cv2.imread("./SKU110K_fixed/images/test/test_0.jpg")
img = detect(net, img, pre_process, post_process, labels)

Image.fromarray(img[:,:,::-1])